In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!git clone https://github.com/NACAUTFPR/seu-repositorio.git

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# O ranking de Cavalos é composto das categorias CN6, CN7 e CN8.  Os resultados de cada categoria é salva no arquivo CavalosNovos.xlsx que deve ser criado ou
# atualizado a cada execução.  O arquivo CavalosNovos.xlsx com as planilhas ranking6anos, ranking7anos e ranking8anos.
# Cada uma dessas planilhas é composta das seguintes colunas: Classificação, Nome do Cavalo, Registro ABCCH, Pontos.

import pandas as pd
import os
import importlib
import subprocess
from openpyxl import load_workbook  # Importe a função load_workbook do pacote openpyxl



# Supondo que você já tenha os DataFrames df_criador e df_garanhão definidos

def criar_planilha(nome_arquivo, nome_planilha, df):
    if not os.path.exists(nome_arquivo):
        # Se o arquivo não existe, cria um novo arquivo Excel e adiciona o DataFrame como uma planilha
        df.to_excel(nome_arquivo, index=False, sheet_name=nome_planilha, engine='openpyxl')
    else:
        # Se o arquivo existe, carrega o arquivo com o openpyxl em modo de leitura-escrita
        workbook = load_workbook(nome_arquivo, read_only=False)

        # Verifica se a planilha já existe no arquivo
        if nome_planilha in workbook.sheetnames:
            # Se a planilha já existe, remove-a antes de criar a nova planilha
            workbook.remove(workbook[nome_planilha])

        # Verifica se pelo menos uma planilha está visível
        if not workbook.sheetnames:
            # Se nenhuma planilha estiver visível, torna a primeira planilha visível
            workbook.active = 0

        # Salva as alterações no arquivo antes de fechá-lo
        workbook.save(nome_arquivo)
        workbook.close()

        # Reabre o arquivo com o pd.ExcelWriter para adicionar a nova planilha
        with pd.ExcelWriter(nome_arquivo, mode='a', engine='openpyxl') as writer:
            df.to_excel(writer, index=False, sheet_name=nome_planilha)


# Associa os nomes dos Cavalos das etapas com o registro BH e cria os ranking por categorias

# Tenta importar o pacote openpyxl
try:
    importlib.import_module('openpyxl')
    openpyxl_installed = True
except ImportError:
    openpyxl_installed = False

# Se o pacote openpyxl não estiver instalado, realiza a instalação
if not openpyxl_installed:
    try:
        subprocess.run(['pip', 'install', 'openpyxl'], check=True)
        print("O pacote openpyxl foi instalado.")
    except subprocess.CalledProcessError:
        print("Não foi possível instalar o pacote openpyxl. Verifique sua conexão com a internet e tente instalar manualmente.")
        exit()

# Mudar para o diretório /content/drive/MyDrive/job/CavalosNovos
os.chdir("/content/drive/MyDrive/job/RankingCavalosNovos")

# Verificar o diretório atual para confirmar se a mudança ocorreu corretamente
current_directory = os.getcwd()
print("Diretório Corrente:", current_directory)

#Os arquivos excel da cada etapa contem os todos os resultados das provas realizadas
etapas = ['CSN OPEN.xlsx', 'CSN CURITIBA.xlsx', 'CSN DMAIO.xlsx', \
          'CSN COPA SP.xlsx', 'CSN COPA JK.xlsx']

# A lista categorias contém a relação de cada uma das categorias
categorias = ['CN6', 'CN7'] #, 'CN8']  # Adicione outras categorias se necessário

arquivorakingcavalosNovos = 'CavalosNovos.xlsx'

colunas_desejadas = ['Classificação','Nome do Cavalo', 'Pontuação final']
arquivo_ABCCH = 'ABCCH.xlsx'

# Ler o arquivo "ABCCH.xlsx"
df_abcch = pd.read_excel("../ABCCH.xlsx")
#print (df_abcch)

df_criadores = [] # Lista para armazenar os dados de cada categoria
df_etapas_categorias = []  # Crie uma lista para armazenar os DataFrames de todas as categorias
# Criar um novo arquivo Excel para salvar os resultados
with pd.ExcelWriter(arquivorakingcavalosNovos, mode='w') as writer:
  # Adicione planilhas vazias para cada categoria
  for categoria in categorias:
        df_vazio = pd.DataFrame()
        planilha = 'ranking' + categoria
        df_vazio.to_excel(writer, sheet_name=planilha, index=False)

  for categoria in categorias:
    df_etapas = [] # Lista para armazenar os dados de cada etapas
    for etapa in etapas:
      #print (df_etapas)
      df_etapa = pd.read_excel(etapa, sheet_name=categoria, usecols=colunas_desejadas)
      # Adiciona a coluna de Categoria com o valor da categoria atual
      df_etapas.append(df_etapa)
      df_etapa['Categoria'] = categoria
      df_etapa['Etapa']= etapa
      df_etapa['Etapa'] = etapa[:-5]
      # Adiciona a coluna Resultado com os valores de Classificação e Pontuação final agregados
      df_etapa ['Resultado'] = df_etapa['Classificação'].astype(str) + '/' + df_etapa['Pontuação final'].astype(str)
      # Mesclar o DataFrame df_etapa com base na coluna "Nome do Cavalo" e a coluna "Cavalo" do df_abcch
      df_etapa = pd.merge(df_etapa, df_abcch[['Cavalo Prova', 'Cavalo ABCCH', 'Registro','Criador', 'Pai', 'Mãe']],
                            left_on='Nome do Cavalo', right_on='Cavalo Prova', how='left')
      df_etapas.append(df_etapa)
    #print (df_etapas)

    #Concatenar os DataFrames de todas as etapas para a categoria atual
    #df_etapas_completa = pd.concat(df_etapas)
    #print (df_etapas_completa)

    df_etapas=pd.concat(df_etapas)
    #print (df_etapas)


    # Agrupar os resultados pelo nome do cavalo e somar a pontuação total de todas as etapas para cada cavalo
    df_etapas = df_etapas.groupby(['Cavalo ABCCH', 'Registro', 'Criador','Pai', 'Mãe'])['Pontuação final'].sum().reset_index()

    # Criando a coluna "Classificação" com base na "Pontuação Garanhão" apenas nas linhas válidas
    df_etapas['Classificação'] = df_etapas['Pontuação final'].rank(ascending=False, method='min')

    # Ordenar o DataFrame resultante pela pontuação total em ordem decrescente
    df_etapas = df_etapas.sort_values('Pontuação final', ascending=False)
    #print (df_etapas)

    # Formatar a coluna "Classificação" como string com o "°" para representar a classificação
    df_etapas['Classificação'] = df_etapas['Classificação'].astype(int).astype(str) + '°'

    # Ordenar o DataFrame resultante pela pontuação total em ordem decrescente
    df_etapas = df_etapas.sort_values('Pontuação final', ascending=False)

    # Reordenando as colunas com a coluna "Classificação" na primeira posição
    df_etapas = df_etapas[['Classificação', 'Cavalo ABCCH', 'Registro', 'Criador','Pai', 'Mãe','Pontuação final']]

    # Renomear a coluna "Pontuação final" para "Pontos"
    df_etapas = df_etapas.rename(columns={'Pontuação final': 'Pontos'})
    planilha='ranking'+categoria
    #print (planilha)
    df_etapas.to_excel(writer, sheet_name=planilha, index=False)
    #print (df_etapas)
    # Adicione o DataFrame df_etapas à lista df_etapas_categorias

#print (df_etapas_completa)

nome_arquivo = 'CavalosNovos.xlsx'
nome_planilhas = ['rankingCN6', 'rankingCN7']
colunas_desejadas = ['Criador','Pai','Pontos']

df_criador = pd.DataFrame() # Crie um DataFrame vazio para armazenar criadores
df_garanhão = pd.DataFrame() # Cria um DataFrame vazio para armazenar garanhões

for nome_planilha in nome_planilhas:
  df = pd.read_excel(nome_arquivo, sheet_name=nome_planilha, usecols=colunas_desejadas)
  df_criador = pd.concat([df_criador,df])
  df_garanhão = pd.concat([df_garanhão,df])

# Agrupar os resultados pelo nome do criador e somar a pontuação total de todas as etapas para cada criador
df_criador = df_criador.groupby(['Criador'])['Pontos'].agg(Pontos='sum', N_Cavalos='size').reset_index()
df_garanhão = df_garanhão.groupby(['Pai'])['Pontos'].agg(Pontos='sum', N_Cavalos='size').reset_index()
#df_garanhão = df_garanhão.groupby(['Pai'])['Pontos'].sum().reset_index()

# Ordenar o DataFrame resultante pela pontuação total em ordem decrescente

# Criando a coluna "Classificação" com base na "Pontuação Garanhão" apenas nas linhas válidas
df_criador ['Classificação'] = df_criador['Pontos'].rank(ascending=False, method='min')
df_garanhão ['Classificação'] = df_garanhão['Pontos'].rank(ascending=False, method='min')

# Ordenar o DataFrame resultante pela pontuação total em ordem decrescente
df_criador = df_criador.sort_values('Pontos', ascending=False)
df_garanhão = df_garanhão.sort_values ('Pontos', ascending=False)
#print (df_etapas)

# Formatar a coluna "Classificação" como string com o "°" para representar a classificação
df_criador['Classificação'] = df_criador['Classificação'].astype(int).astype(str) + '°'
df_garanhão['Classificação'] = df_garanhão['Classificação'].astype(int).astype(str) + '°'

# Reordenando as colunas com a coluna "Classificação" na primeira posição
df_criador = df_criador [['Classificação', 'Criador','N_Cavalos', 'Pontos']]
df_garanhão = df_garanhão [['Classificação', 'Pai', 'N_Cavalos', 'Pontos']]

criar_planilha(nome_arquivo, 'RankingCriador', df_criador)
criar_planilha(nome_arquivo, 'RankingGaranhão', df_garanhão)


#print (df_criador)
#print (df_garanhão)

print("Os DataFrames foram adicionados como planilhas no arquivo Excel com sucesso!")

arquivorakingcavalosNovos = 'CavalosNovos.xlsx'

colunas_desejadas = ['Classificação','Nome do Cavalo', 'Pontuação final']
arquivo_ABCCH = 'ABCCH.xlsx'

df_categorias = []  # Lista para armazenar os dados de cada categoria

for categoria in categorias:
    df_etapas = pd.DataFrame() #[]  # Lista para armazenar os dados de cada etapa
    df_etapas_completa = pd.DataFrame() # []
    for etapa in etapas:
        df_etapa = pd.read_excel(etapa, sheet_name=categoria, usecols=colunas_desejadas)
        df_etapa['Categoria'] = categoria
        df_etapa['Etapa'] = etapa[:-5]
        df_etapa['Resultado'] = df_etapa['Classificação'].astype(str) + '/' + df_etapa['Pontuação final'].astype(str)
        df_etapa = pd.merge(df_etapa, df_abcch[['Cavalo Prova', 'Cavalo ABCCH', 'Registro', 'Criador', 'Pai', 'Mãe']],
                            left_on='Nome do Cavalo', right_on='Cavalo Prova', how='left')
        #df_etapas.append(df_etapa)
        df_etapas = pd.concat([df_etapas, df_etapa.copy()])

    #df_etapas = pd.concat(df_etapas)
    #df_etapas_completa = pd.concat(df_etapas_completa)
    #df_categorias.append(df_etapas)
    #df_categorias.append(df_etapas_completa)
    df_categorias.append(df_etapas.copy())

df_resultado_completo = pd.concat(df_categorias)
#print (df_resultado_completo)

# Eliminar linhas duplicadas com base em todas as colunas do DataFrame
#df_resultado_completo = df_resultado_completo.drop_duplicates()

#print (df_resultado_completo)

df_resultado_completo = df_resultado_completo.rename(columns={'Pontuação final': 'Pontuação da Etapa'})

# Agrupar os resultados pelo nome do cavalo e somar a pontuação da etapa para cada cavalo
df_pontos_etapa = df_resultado_completo.groupby('Cavalo ABCCH')['Pontuação da Etapa'].sum().reset_index()

# Renomear a coluna 'Pontuação da Etapa' para 'Pontos'
df_pontos_etapa.rename(columns={'Pontuação da Etapa': 'Pontos'}, inplace=True)

# Mesclar os DataFrames para adicionar a coluna 'Pontos' no df_resultado_completo
df_resultado_completo = pd.merge(df_resultado_completo, df_pontos_etapa, on='Cavalo ABCCH', how='left')

# Ordenar o DataFrame em ordem decrescente pela coluna "Pontuação da Prova"
df_resultado_completo = df_resultado_completo.sort_values(by='Pontos', ascending=False)

# Selecionar apenas os resultados da categoria CN6
df_resultado_CN6 = df_resultado_completo[df_resultado_completo['Categoria'] == 'CN6']

# Reordenando as colunas com a coluna "Classificação" na primeira posição
df_resultado_CN6 = df_resultado_CN6 [['Cavalo ABCCH', 'Registro','Categoria', 'Etapa','Resultado', 'Pontos']]
df_resultado_CN7 = df_resultado_completo[df_resultado_completo['Categoria'] == 'CN7']
df_resultado_CN7 = df_resultado_CN7 [['Cavalo ABCCH', 'Registro','Categoria', 'Etapa','Resultado', 'Pontos']]
#df_resultado_CN8 = df_resultado_completo[df_resultado_completo['Categoria'] == 'CN8']
#df_resultado_CN8 = df_resultado_CN8 [['Cavalo ABCCH', 'Registro','Categoria', 'Etapa','Resultado', 'Pontos']]


#print (df_resultado_CN6)
#print (df_resultado_CN7)


criar_planilha(arquivorakingcavalosNovos, 'ResultadoCompletoCN6', df_resultado_CN6)
criar_planilha(arquivorakingcavalosNovos, 'ResultadoCompletoCN7', df_resultado_CN7)



Diretório Corrente: /content/drive/MyDrive/job/RankingCavalosNovos
Os DataFrames foram adicionados como planilhas no arquivo Excel com sucesso!


In [ ]:
# Associa os nomes dos Cavalos das etapas com o registro BH e cria os ranking por categorias
import pandas as pd
import os
import importlib
import subprocess
#from openpyxl import load_workbook

# Tenta importar o pacote openpyxl
try:
    importlib.import_module('openpyxl')
    openpyxl_installed = True
except ImportError:
    openpyxl_installed = False

# Se o pacote openpyxl não estiver instalado, realiza a instalação
if not openpyxl_installed:
    try:
        subprocess.run(['pip', 'install', 'openpyxl'], check=True)
        print("O pacote openpyxl foi instalado.")
    except subprocess.CalledProcessError:
        print("Não foi possível instalar o pacote openpyxl. Verifique sua conexão com a internet e tente instalar manualmente.")
        exit()

# Mudar para o diretório /content/drive/MyDrive/job/CavalosNovos
os.chdir("/content/drive/MyDrive/job/RankingCavalosNovos")

# Verificar o diretório atual para confirmar se a mudança ocorreu corretamente
current_directory = os.getcwd()
print("Diretório Corrente:", current_directory)

nome_arquivo = 'CavalosNovos.xlsx'
nome_planilhas = ['rankingCN6', 'rankingCN7']
colunas_desejadas = ['Criador','Pai','Pontos']

#Os arquivos excel da cada etapa contem os todos os resultados das provas realizadas
etapas = ['CSN OPEN.xlsx', 'CSN CURITIBA.xlsx', 'CSN DMAIO.xlsx', \
          'CSN COPA SP.xlsx', 'CSN COPA JK.xlsx']

# A lista categorias contém a relação de cada uma das categorias
categorias = ['CN6', 'CN7'] #, 'CN8']  # Adicione outras categorias se necessário

arquivorakingcavalosNovos = 'CavalosNovos.xlsx'

colunas_desejadas = ['Classificação','Nome do Cavalo', 'Pontuação final']
arquivo_ABCCH = 'ABCCH.xlsx'

# Ler o arquivo "ABCCH.xlsx"
df_abcch = pd.read_excel("../ABCCH.xlsx")
#print (df_abcch)

df_criadores = [] # Lista para armazenar os dados de cada categoria
df_etapas_categorias = []  # Crie uma lista para armazenar os DataFrames de todas as categorias
# Criar um novo arquivo Excel para salvar os resultados
with pd.ExcelWriter(arquivorakingcavalosNovos, mode='w') as writer:
  # Adicione planilhas vazias para cada categoria
  for categoria in categorias:
        df_vazio = pd.DataFrame()
        planilha = 'ranking' + categoria
        df_vazio.to_excel(writer, sheet_name=planilha, index=False)

  for categoria in categorias:
    df_etapas = [] # Lista para armazenar os dados de cada etapas
    for etapa in etapas:
      #print (df_etapas)
      df_etapa = pd.read_excel(etapa, sheet_name=categoria, usecols=colunas_desejadas)
      # Adiciona a coluna de Categoria com o valor da categoria atual
      df_etapas.append(df_etapa)
      df_etapa['Categoria'] = categoria
      df_etapa['Etapa']= etapa
      df_etapa['Etapa'] = etapa[:-5]
      # Adiciona a coluna Resultado com os valores de Classificação e Pontuação final agregados
      df_etapa ['Resultado'] = df_etapa['Classificação'].astype(str) + '/' + df_etapa['Pontuação final'].astype(str)
      # Mesclar o DataFrame df_etapa com base na coluna "Nome do Cavalo" e a coluna "Cavalo" do df_abcch
      df_etapa = pd.merge(df_etapa, df_abcch[['Cavalo Prova', 'Cavalo ABCCH', 'Registro','Criador', 'Pai', 'Mãe']],
                            left_on='Nome do Cavalo', right_on='Cavalo Prova', how='left')
      df_etapas.append(df_etapa)
    #print (df_etapas)

    #Concatenar os DataFrames de todas as etapas para a categoria atual
    #df_etapas_completa = pd.concat(df_etapas)
    #print (df_etapas_completa)

    df_etapas=pd.concat(df_etapas)
    #print (df_etapas)


    # Agrupar os resultados pelo nome do cavalo e somar a pontuação total de todas as etapas para cada cavalo
    df_etapas = df_etapas.groupby(['Cavalo ABCCH', 'Registro', 'Criador','Pai', 'Mãe'])['Pontuação final'].sum().reset_index()

    # Criando a coluna "Classificação" com base na "Pontuação Garanhão" apenas nas linhas válidas
    df_etapas['Classificação'] = df_etapas['Pontuação final'].rank(ascending=False, method='min')

    # Ordenar o DataFrame resultante pela pontuação total em ordem decrescente
    df_etapas = df_etapas.sort_values('Pontuação final', ascending=False)
    #print (df_etapas)

    # Formatar a coluna "Classificação" como string com o "°" para representar a classificação
    df_etapas['Classificação'] = df_etapas['Classificação'].astype(int).astype(str) + '°'

    # Ordenar o DataFrame resultante pela pontuação total em ordem decrescente
    df_etapas = df_etapas.sort_values('Pontuação final', ascending=False)

    # Reordenando as colunas com a coluna "Classificação" na primeira posição
    df_etapas = df_etapas[['Classificação', 'Cavalo ABCCH', 'Registro', 'Criador','Pai', 'Mãe','Pontuação final']]

    # Renomear a coluna "Pontuação final" para "Pontos"
    df_etapas = df_etapas.rename(columns={'Pontuação final': 'Pontos'})
    planilha='ranking'+categoria
    #print (planilha)
    df_etapas.to_excel(writer, sheet_name=planilha, index=False)
    #print (df_etapas)
    # Adicione o DataFrame df_etapas à lista df_etapas_categorias

#print (df_etapas_completa)

nome_arquivo = 'CavalosNovos.xlsx'
nome_planilhas = ['rankingCN6', 'rankingCN7']
colunas_desejadas = ['Criador','Pai','Pontos']

df_criador = pd.DataFrame() # Crie um DataFrame vazio para armazenar criadores
df_garanhão = pd.DataFrame() # Cria um DataFrame vazio para armazenar garanhões

for nome_planilha in nome_planilhas:
  df = pd.read_excel(nome_arquivo, sheet_name=nome_planilha, usecols=colunas_desejadas)
  df_criador = pd.concat([df_criador,df])
  df_garanhão = pd.concat([df_garanhão,df])

# Agrupar os resultados pelo nome do criador e somar a pontuação total de todas as etapas para cada criador
df_criador = df_criador.groupby(['Criador'])['Pontos'].agg(Pontos='sum', N_Cavalos='size').reset_index()
df_garanhão = df_garanhão.groupby(['Pai'])['Pontos'].agg(Pontos='sum', N_Cavalos='size').reset_index()
#df_garanhão = df_garanhão.groupby(['Pai'])['Pontos'].sum().reset_index()

# Ordenar o DataFrame resultante pela pontuação total em ordem decrescente

# Criando a coluna "Classificação" com base na "Pontuação Garanhão" apenas nas linhas válidas
df_criador ['Classificação'] = df_criador['Pontos'].rank(ascending=False, method='min')
df_garanhão ['Classificação'] = df_garanhão['Pontos'].rank(ascending=False, method='min')

# Ordenar o DataFrame resultante pela pontuação total em ordem decrescente
df_criador = df_criador.sort_values('Pontos', ascending=False)
df_garanhão = df_garanhão.sort_values ('Pontos', ascending=False)
#print (df_etapas)

# Formatar a coluna "Classificação" como string com o "°" para representar a classificação
df_criador['Classificação'] = df_criador['Classificação'].astype(int).astype(str) + '°'
df_garanhão['Classificação'] = df_garanhão['Classificação'].astype(int).astype(str) + '°'

# Reordenando as colunas com a coluna "Classificação" na primeira posição
df_criador = df_criador [['Classificação', 'Criador','N_Cavalos', 'Pontos']]
df_garanhão = df_garanhão [['Classificação', 'Pai', 'N_Cavalos', 'Pontos']]

criar_planilha(nome_arquivo, 'RankingCriador', df_criador)
criar_planilha(nome_arquivo, 'RankingGaranhão', df_garanhão)


#print (df_criador)
#print (df_garanhão)

print("Os DataFrames foram adicionados como planilhas no arquivo Excel com sucesso!")


Diretório Corrente: /content/drive/MyDrive/job/RankingCavalosNovos


In [ ]:
# Cria os Rankings Criadores e Garanhão
import pandas as pd
import os
import importlib
import subprocess
#from openpyxl import load_workbook

# Tenta importar o pacote openpyxl
try:
    importlib.import_module('openpyxl')
    openpyxl_installed = True
except ImportError:
    openpyxl_installed = False

# Se o pacote openpyxl não estiver instalado, realiza a instalação
if not openpyxl_installed:
    try:
        subprocess.run(['pip', 'install', 'openpyxl'], check=True)
        print("O pacote openpyxl foi instalado.")
    except subprocess.CalledProcessError:
        print("Não foi possível instalar o pacote openpyxl. Verifique sua conexão com a internet e tente instalar manualmente.")
        exit()

# Mudar para o diretório /content/drive/MyDrive/job/CavalosNovos
os.chdir("/content/drive/MyDrive/job/RankingCavalosNovos")

# Verificar o diretório atual para confirmar se a mudança ocorreu corretamente
current_directory = os.getcwd()
print("Diretório Corrente:", current_directory)

nome_arquivo = 'CavalosNovos.xlsx'
nome_planilhas = ['rankingCN6', 'rankingCN7']
colunas_desejadas = ['Criador','Pai','Pontos']

df_criador = pd.DataFrame() # Crie um DataFrame vazio para armazenar criadores
df_garanhão = pd.DataFrame() # Cria um DataFrame vazio para armazenar garanhões

for nome_planilha in nome_planilhas:
  df = pd.read_excel(nome_arquivo, sheet_name=nome_planilha, usecols=colunas_desejadas)
  df_criador = pd.concat([df_criador,df])
  df_garanhão = pd.concat([df_garanhão,df])

# Agrupar os resultados pelo nome do criador e somar a pontuação total de todas as etapas para cada criador
df_criador = df_criador.groupby(['Criador'])['Pontos'].agg(Pontos='sum', N_Cavalos='size').reset_index()
df_garanhão = df_garanhão.groupby(['Pai'])['Pontos'].agg(Pontos='sum', N_Cavalos='size').reset_index()
#df_garanhão = df_garanhão.groupby(['Pai'])['Pontos'].sum().reset_index()

# Ordenar o DataFrame resultante pela pontuação total em ordem decrescente

# Criando a coluna "Classificação" com base na "Pontuação Garanhão" apenas nas linhas válidas
df_criador ['Classificação'] = df_criador['Pontos'].rank(ascending=False, method='min')
df_garanhão ['Classificação'] = df_garanhão['Pontos'].rank(ascending=False, method='min')

# Ordenar o DataFrame resultante pela pontuação total em ordem decrescente
df_criador = df_criador.sort_values('Pontos', ascending=False)
df_garanhão = df_garanhão.sort_values ('Pontos', ascending=False)
#print (df_etapas)

# Formatar a coluna "Classificação" como string com o "°" para representar a classificação
df_criador['Classificação'] = df_criador['Classificação'].astype(int).astype(str) + '°'
df_garanhão['Classificação'] = df_garanhão['Classificação'].astype(int).astype(str) + '°'

# Reordenando as colunas com a coluna "Classificação" na primeira posição
df_criador = df_criador [['Classificação', 'Criador','N_Cavalos', 'Pontos']]
df_garanhão = df_garanhão [['Classificação', 'Pai', 'N_Cavalos', 'Pontos']]

criar_planilha(nome_arquivo, 'RankingCriador', df_criador)
criar_planilha(nome_arquivo, 'RankingGaranhão', df_garanhão)


#print (df_criador)
#print (df_garanhão)

print("Os DataFrames foram adicionados como planilhas no arquivo Excel com sucesso!")



Diretório Corrente: /content/drive/MyDrive/job/RankingCavalosNovos
Os DataFrames foram adicionados como planilhas no arquivo Excel com sucesso!


In [ ]:
import pandas as pd
import os
import importlib
import subprocess

#Os arquivos excel da cada etapa contem os todos os resultados das provas realizadas
etapas = ['CSN OPEN.xlsx', 'CSN CURITIBA.xlsx', 'CSN DMAIO.xlsx', \
          'CSN COPA SP.xlsx', 'CSN COPA JK.xlsx']

# A lista categorias contém a relação de cada uma das categorias
categorias = ['CN6', 'CN7'] #, 'CN8']  # Adicione outras categorias se necessário

arquivorakingcavalosNovos = 'CavalosNovos.xlsx'

colunas_desejadas = ['Classificação','Nome do Cavalo', 'Pontuação final']
arquivo_ABCCH = 'ABCCH.xlsx'

df_categorias = []  # Lista para armazenar os dados de cada categoria

for categoria in categorias:
    df_etapas = pd.DataFrame() #[]  # Lista para armazenar os dados de cada etapa
    df_etapas_completa = pd.DataFrame() # []
    for etapa in etapas:
        df_etapa = pd.read_excel(etapa, sheet_name=categoria, usecols=colunas_desejadas)
        df_etapa['Categoria'] = categoria
        df_etapa['Etapa'] = etapa[:-5]
        df_etapa['Resultado'] = df_etapa['Classificação'].astype(str) + '/' + df_etapa['Pontuação final'].astype(str)
        df_etapa = pd.merge(df_etapa, df_abcch[['Cavalo Prova', 'Cavalo ABCCH', 'Registro', 'Criador', 'Pai', 'Mãe']],
                            left_on='Nome do Cavalo', right_on='Cavalo Prova', how='left')
        #df_etapas.append(df_etapa)
        df_etapas = pd.concat([df_etapas, df_etapa.copy()])

    #df_etapas = pd.concat(df_etapas)
    #df_etapas_completa = pd.concat(df_etapas_completa)
    #df_categorias.append(df_etapas)
    #df_categorias.append(df_etapas_completa)
    df_categorias.append(df_etapas.copy())

df_resultado_completo = pd.concat(df_categorias)
print (df_resultado_completo)

# Eliminar linhas duplicadas com base em todas as colunas do DataFrame
#df_resultado_completo = df_resultado_completo.drop_duplicates()

print (df_resultado_completo)

df_resultado_completo = df_resultado_completo.rename(columns={'Pontuação final': 'Pontuação da Etapa'})

# Agrupar os resultados pelo nome do cavalo e somar a pontuação da etapa para cada cavalo
df_pontos_etapa = df_resultado_completo.groupby('Cavalo ABCCH')['Pontuação da Etapa'].sum().reset_index()

# Renomear a coluna 'Pontuação da Etapa' para 'Pontos'
df_pontos_etapa.rename(columns={'Pontuação da Etapa': 'Pontos'}, inplace=True)

# Mesclar os DataFrames para adicionar a coluna 'Pontos' no df_resultado_completo
df_resultado_completo = pd.merge(df_resultado_completo, df_pontos_etapa, on='Cavalo ABCCH', how='left')

# Ordenar o DataFrame em ordem decrescente pela coluna "Pontuação da Prova"
df_resultado_completo = df_resultado_completo.sort_values(by='Pontos', ascending=False)

# Selecionar apenas os resultados da categoria CN6
df_resultado_CN6 = df_resultado_completo[df_resultado_completo['Categoria'] == 'CN6']

# Reordenando as colunas com a coluna "Classificação" na primeira posição
df_resultado_CN6 = df_resultado_CN6 [['Cavalo ABCCH', 'Registro','Categoria', 'Etapa','Resultado', 'Pontos']]
df_resultado_CN7 = df_resultado_completo[df_resultado_completo['Categoria'] == 'CN7']
df_resultado_CN7 = df_resultado_CN7 [['Cavalo ABCCH', 'Registro','Categoria', 'Etapa','Resultado', 'Pontos']]
#df_resultado_CN8 = df_resultado_completo[df_resultado_completo['Categoria'] == 'CN8']
#df_resultado_CN8 = df_resultado_CN8 [['Cavalo ABCCH', 'Registro','Categoria', 'Etapa','Resultado', 'Pontos']]


print (df_resultado_CN6)
print (df_resultado_CN7)


criar_planilha(arquivorakingcavalosNovos, 'ResultadoCompletoCN6', df_resultado_CN6)
criar_planilha(arquivorakingcavalosNovos, 'ResultadoCompletoCN7', df_resultado_CN7)


print(df_resultado_CN6)


   Classificação          Nome do Cavalo  Pontuação final Categoria  \
0             1º  HSA WENDY DES FLANDRES               29       CN6   
1             2º     REGINA VD DONKHOEVE               27       CN6   
2             3º   VIP JCR ZIMBABWE (IA)               20       CN6   
3             4º             NICOLE JMEN               20       CN6   
4             5º        CORNETA TRV (TE)               19       CN6   
..           ...                     ...              ...       ...   
8             9º            BALOUBELLA G               16       CN7   
9            10º          COLBAN TW (IA)               12       CN7   
10            1º          MISS MOURADA M               12       CN7   
11            1º            MR MOURADA M               12       CN7   
12            1º     CONECTOR SANTA ROSA               12       CN7   

          Etapa Resultado            Cavalo Prova           Cavalo ABCCH  \
0      CSN OPEN     1º/29  HSA WENDY DES FLANDRES     WENDY DES FLANDRE